# Overview

We want to combine the proteins given by the CullPDB PISCES server [link] with the secondary structure information provided by DSSP [link].

## Prerequisites

### Cull PDB: PISCES

Proteins from the PDB can be queried based on criteria such as resolution, sequence identity, etc. It's possible (as of 20/03/2018) to download different lists [here](http://dunbrack.fccc.edu/Guoli/pisces_download.php).

### DSSP files

The PISCES lists provide PDB ID's, but they do not have the secondary structure information. To get that, you need to download the DSSP information from the PDB. This can be done directly [here](http://swift.cmbi.ru.nl/gv/dssp/).

By syncing the database locally, the individual \*.dssp files can be parsed by the script [here](https://gist.github.com/dillondaudert/94785e9cc0318ac69243c6283da3a032).

### Next Steps
The rest of this notebook will assume that a list downloaded from PISCES as well as some number of .csv files containing the parsed DSSP data exist in a `data/dssp` folder.

## Loading the Data

Essentially, we want to do a join on the PDB id field of the PISCES and DSSP datasets. Since these are both in either tab-separated or csv format, Pandas is an ideal candidate for doing this.

In [2]:
import pandas as pd
from pathlib import Path
datadir = str(Path(Path.home(), "data", "dssp"))

In [ ]:
cpdb_df = pd.read_csv(datadir+"/cullpdb_pc30_res2.5_R1.0_d180208_chains15102.txt", delim_whitespace=True)
print(cpdb_df.columns)
print(cpdb_df.iloc[0])

In [ ]:
# Check some assumptions about this data
all_length_5 = all(cpdb_df["IDs"].str.len() == 5)
cpdb_long_ids_df = cpdb_df["IDs"][cpdb_df["IDs"].str.len() != 5]
# All IDs are unique
unique_ids = len(cpdb_df["IDs"]) == len(cpdb_df["IDs"].unique())

In [ ]:
dssp_1_df = pd.read_csv(datadir+"/dssp_1.csv")
print(dssp_1_df.columns)
print(dssp_1_df["dssp_id"][0], dssp_1_df["seq"][0])

In [ ]:
# Check some assumptions about this data
all_length_4 = all(dssp_1_df["dssp_id"].str.len() == 4)
unique_ids = len(dssp_1_df["dssp_id"]) == len(dssp_1_df["dssp_id"].unique())

### Note on CPDB chains and DSSP ids
The PISCES server checks each CHAIN of a PDB entry individually. As such, the cpdb IDs may contain all or only some of the chains of a particular PDB entry. On the other hand, the DSSP outputs a single file / entry per PDB ID, which will include (*I assume*) all of the chains for that entry.

For simplicity, I will assume that if a particular 4-letter PDB entry appears at least once in the cpdb IDs, then the entire entry is acceptable to put in the dataset. This means that I can just take the unique 4-letter IDs from the cpdb_df and do an inner join with the dssp_dfs. Once this is done for all the dssp files, the resulting dataframe will contain the sequences, secondary structure, and other features of the PDB entries specified by the PISCES cull pdb list.

#### Duplicate IDs
We are curious about the relationship between the CPDB chain entries and the DSSP PDB ids. 

If the first 4 characters in a CPDB id are duplicated, then that indicates that more than 1 chain of a PDB entry met the criteria for the culled dataset (seq identity, resolution, etc). If we ignore the 5th character and only pull the first, we will get all chains for each PDB entry; both those that met the criteria and those that didn't. Furthermore, they will be separated by a `!` character.

Ideally, we would split the DSSP entries on `!` in a way that is consistent with the CPDB naming scheme. Is the first chain `A`, the second `B`, etc? This is what I want to find out.

1. Find entries with more than 1 chain in CPDB ids (called duplicates)
2. Compare the sequences in the CPDB file with each chain entry in the DSSP file. Is there a correspondance between the e.g. 1d3bA and the first sequence before the first `!`?

#### Example: duplicate id in cpdb: `1d3b`
- The DSSP file has all chains (verified by looking at this protein on PDB)
- Only the first 2 are marked as "unique"; upon inspection, the last 2 chains seem to be different subsequences of the first 2.
- The CPDB file contains chains `1d3bB` and `1d3bI`...

**IDEA**: If the DSSP files contain all the chains, we can split on `!*`. This would get us exactly the protein chains we are interested in, and reduce a lot of redundancy.

**NOTE**: We would also have to split all the other DSSP fields at the same position... Need a script to do this. 

#### Modifying the parse_dssp script
We can modify this script to also separate based on chains. There are chain identifiers provided, but there appears to be overlap...

`num` vs. `resnum` vs. `chain id`: Example 1d3a chain B residue 54 (A/B/C?). Checking on PDB website, both chain A and B contain what is referred to as "54A" in the dssp file, in this case isoleucine. Should we just ignore any positions that contain more than 1 residue, using the first only?

This script will now split dssp files into their constituent chains, appending the chain id to the end of the dssp_id, and creating records for each chain. Some edges cases where the parser incorrectly identifies the chain id exist, but those are skipped.

#### Modifying DSSPData
The parse function had to be changed, since `resnum` occupied characters 5:11 and `moltyp` only occupied a single character indicating the chain in column 11.

## Split DSSP records into constituent chains
Each DSSP PDB id consists of all the chains for that entry. We need to split on `*`. See the `parse_dssp.py` file.

Currently, assume that the number of chain identifiers found will equal the number of times `!*` occurs, plus 1. If this isn't met, just skip for now.

## Joining the Data

We want to concatenate the two datasets, joining on the two id's. Since the cpdb data is a subset of the dssp data, we join on the cpdb id field (after taking the first 4 characters only)

In [ ]:
# get a dataframe consisting of only the first 4 characters of each cpdb id
cpdb_ids = cpdb_df["IDs"].str[0:4].str.lower()
print(cpdb_ids.loc[0:5])
# check if these are still unique
print(len(cpdb_ids) == len(cpdb_ids.unique()))
# only take the unique entries
cpdb_ids = cpdb_ids[~cpdb_ids.duplicated()]
print(cpdb_ids.is_unique)

# get the cpdb dataset with only unique, 4-letter IDs
cpdb_df_unique = cpdb_df[~cpdb_df["IDs"].duplicated()]

cpdb_df_unique["dssp_id"] = cpdb_ids
cpdb_df_unique.drop(labels=["IDs"], axis=1)

In [ ]:
# merge the two datasets
merged_1 = dssp_1_df.merge(cpdb_df_unique, how="inner", on="dssp_id")
print(merged_1.columns)
print(len(merged_1))

In [ ]:
# Now do this for all dssp files
merged_frames = []
for i in range(1,12):
    dssp_df = pd.read_csv(datadir+"/dssp_%d.csv" % i)
    merged = dssp_df.merge(cpdb_df_unique, how="inner", on="dssp_id")
    print(len(merged))
    merged_frames.append(merged)

In [ ]:
all_merged = pd.concat(merged_frames)
print(len(all_merged))
print(all_merged["dssp_id"].is_unique)
# Write out the sequence and secondary structure to a file
all_merged[["dssp_id", "seq", "ss"]].to_csv(datadir+"/cpdb_dssp_%d.csv" % len(all_merged), index=False)

## PSIBLAST

Similar to CPDB, we will calculate position-specific profile similarity scores using PSI-BLAST. The process is as follows:
- Download the UniRef90 database, and filter using [pfilt](http://bioinf.cs.ucl.ac.uk/psipred/) to remove low information content and coiled-coil regions.
- Create a BLAST database out of the filtered sequences in FASTA format using the blast command line tool
- Run PSI-BLAST on the CullPDB dataset downloaded via PISCES against the BLAST database just created, with an inclusion threshold of 0.001 for 3 iterations, and transform the profile scores into the range [0, 1) via logistic sigmoid

These scores will be saved, along with the one-hot encoding of the amino acid at each position, as a numpy vector.

## UniRef90

The UniRef90 dataset can be downloaded from [uniprot.org](http://www.uniprot.org/uniref/?query=&fil=identity:0.9). Note this is a very large download...

## pfilt

See the [README](http://bioinfadmin.cs.ucl.ac.uk/downloads/pfilt/).


# Creating a dataset without PSIBLAST

## Creating .TFRecords files

We could use the string pairs directly as inputs to a learning model. The TensorFlow tf.data API allows for reading text data and converting to feature vectors as a preprocessing step in a model. However, this places a heavy computational bottleneck at the CPU that could slow down training. Since the dataset is ~14k sequences, we can save them directly as feature vectors in a .tfrecords file that is loaded into memory at training.

## Features

Oftentimes, position-specific features are calculated using PSIBLAST or hidden markov models. To keep things simple at this stage, we can simply append feature vectors that correspond to the amino acids / secondary structures of the sequence and save those as TF records. The features are the following:

In [3]:
aa_feats = pd.read_csv("./cpdb2_aa_features.csv", index_col=0)
aa_feats

,A,C,D,E,F,G,H,I,K,L,...,X,!,SOS,EOS,hydrophobicity,polar,hydropathy intensity,hydrophilicity,pH_l,vdW_vol
A,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.8,3.0,6.01,67.0
C,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,-1.0,2.5,-1.0,5.05,86.0
D,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,1.0,-3.5,3.0,2.85,91.0
E,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,1.0,-3.5,3.0,3.15,109.0
F,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,-1.0,2.8,-2.5,5.49,135.0
G,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.4,0.0,6.06,48.0
H,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-1.0,1.0,-3.2,-0.5,7.60,118.0
I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,-1.0,4.5,-1.8,6.05,124.0
K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,2.0,1.0,-3.9,3.0,9.60,135.0
L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,-1.0,3.8,-1.8,6.01,124.0


In [4]:
ss_feats = pd.read_csv("./cpdb2_ss_features.csv", index_col=0)
ss_feats

,H,B,E,G,I,T,S,U,SOS,EOS
labels,,,,,,,,,,
H,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
G,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
I,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
T,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
S,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
U,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
SOS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


The secondary structures are the targets, and so their "features" are simply one-hot encodings of the letters, including special SOS/EOS tokens.
The amino acid features include a one-hot encoding the 20 proteinogenic amino acids as well as six physicochemical properties: hydrophobicity, polarity, hydropathy intensity, hydrophilicity, isoelectric point, and van der Waals volume. 

See the [make_tfrecords.py](./make_tfrecords.py) script for how the protein sequences are processed into tf records.

These files are read into TensorFlow models using the tf.data API.

### Note on cpdb2 sequences with the character 'b', and 'j', and 'o', and 'u', and 'z'
I haven't figured out what this means, but they are fairly common. 'b' is more common than 'j'. For the moment, I am just replacing them with "X".